In [34]:
# pythonic version of the bash script instructions:
# for link to bash instructions, go to:
# https://gate.nmr.mgh.harvard.edu/wiki/bradmci/index.php/Individual_Reconstruction_%26_Analysis

# TO DO:
# figure out subname problem .. symlink?
# enable: "issue a new project (each project has tasks connected to them__ BUT THESE TASKS COULD HAVE DIFFERENT PARAMS ETC>)"

In [35]:
def parse_findsession(findsession_output):
    session_values = {}
    for elements in findsession_output:
        try:
            [a, b] = elements.split(": ")
            a = a.strip()
            b = b.strip()
            if a in session_values:
                session_values[a].append(b)
            else:
                session_values[a]=[b]
        except ValueError:
            continue
    return session_values

In [36]:
def copyanything(src, dst):
    try:
        shutil.copytree(src, dst)
    except OSError as exc:
        if exc.errno == errno.ENOTDIR:
            shutil.copy(src, dst)
        else: raise

In [37]:
def maketxt(name, location):
    f = open(location+"subjectname", "w+")
    f.write(name)
    f.close()

In [38]:
def verifyrun(question):
    sys.stdout.write('%s [y/n]: ' %question)
    while True:
        try:
            return strtobool(input().lower())
        except ValueError:
            sys.stdout.write('Please respond with \'y\' or \'n\'.\n')
    

In [48]:
# load in all modules and pathways
from distutils.util import strtobool
import sys
import os
from os import system
from os.path import abspath

from nipype import Workflow, Node, MapNode, Function
from nipype.interfaces.fsl import BET, IsotropicSmooth, ApplyMask
from nipype.interfaces.freesurfer import MRIConvert

from nilearn.plotting import plot_anat
#%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import csv
import re
import shutil, errno

home = '/autofs/homes/002/rje11/'
mydirectory = '/autofs/cluster/animal/users/rezzo/'

####### input needed to run script: #######

# TASK: name this after task string in scaninfo file (e.g. 'FPW' or'MT'. 
# this will be the title of the bold directory and will include runs that share 
# sequencename, aquisition dimesions & resolution, and TR.
task = ['FPW', 'MT'] 

# name of recon (e.g. edit.001.056). Can I import this from redcap?
subname = '' 

# paradigm file (might have to be manual)

unpackedtype = 'nii'

In [58]:
subjectlist =  ['PCA_SKRA_M_48']  #['PCA_ITAY_F_34_201701220']

for indiv in subjectlist:

    findsession_output = !findsession PCA_SKRA_M_48
    cmdstring = 'findsession %s' % (indiv)
    system(cmdstring)
    print(cmdstring)

    # create dictionary with listed values (corresponding to each session)
    #session_values = parse_findsession(findsession_output)
    session_values = parse_findsession(findsession_output)
    
    find = re.compile(r"^[^_]*")
    projectname = re.search(find, indiv).group(0)
    
    try:
        # create a directory for unique session
        projectdir = mydirectory+projectname
        #projectdir = mydirectory+'projectname' # override with this until I know nomenclature
        os.makedirs(projectdir)
    except FileExistsError:
        print('PROJECT directory exists')

findsession PCA_SKRA_M_48
PROJECT directory exists


In [60]:
session_values

{'SUBJECT': ['PCA_SKRA_M_48_20171018'],
 'SUBJ ID': ['17.10.18-13:34:58-DST-1.3.12.2.1107.5.2.32.35162'],
 'DATE': ['October 18, 2017'],
 'TIME': ['14:19:01'],
 'OPERATR': ['hsp13:pca (bay3) [D]'],
 'XPRMNTR': [''],
 'PATH': ['/cluster/archive/319/siemens/TrioTim-35162-20171018-133630-001486']}

In [61]:
# for each session (5-10 min PER RUN)
for elements in range(0, len(session_values["SUBJECT"])): # define range as number of sessions
    srcdir = session_values["PATH"][elements]+"/"
    SESSION_DIR = projectdir+"/"+session_values["SUBJECT"][elements]+'/'
    dicomdir = SESSION_DIR+"dicoms/"
    taskname = session_values["SUBJECT"][elements]
    anatdir = SESSION_DIR+"anat/"
    
    try:
        # make directories for each session if not already existing
        os.makedirs(anatdir)
        os.makedirs(dicomdir)

        #load in the anatomical data\figure out best way to do this !!!!!!!!!!!!!
        anat_src = '/autofs/cluster/animal/users/rezzo/recons/'+ session_values["SUBJECT"][elements][4:] # temporary
        #recon = '/autofs/cluster/animal/scan_data/dl-xnat/MASTER/ITAY_F_34/PCA_ITAY_F_34_20171220/recons/FS6_004_006/2017-12-21T17-10-11-115/edit.FS6_004_006'
        cmdstring = "cp -a %s/* %s" % (anat_src, anatdir)
        system(cmdstring)
        print(cmdstring)

        # put subjectname textfile into main folder !!!!!!!!!!!!!!!!!!
        maketxt(subname, SESSION_DIR)

    except FileExistsError:
        print("directory already exists, proceeding with unpacking")
        system('echo ------directory already exists, proceeding with unpacking------')

    #make temp directory and unpack first dicom to chck if exists
    cmdstring = 'mkdir %stmpdir'% (SESSION_DIR)
    system(cmdstring)
    firstfile = os.listdir(srcdir)[0]
    cmdstring = "rsync %s %s" % (srcdir+firstfile, SESSION_DIR+'tmpdir/')
    system(cmdstring)
    namecheck = os.listdir(SESSION_DIR+'tmpdir/')[0]

    #check if file exists in dicom directory (if one is there that means all 
    # scans have been unpacked: bold, MEMPRAGE, etc.)
    print(namecheck)
    if os.path.isfile(dicomdir+namecheck):
        print('dicoms appear to be unpacked already')
        system('echo --------dicoms appear to be unpacked already.. proceeding--------')
        # delete tmpdir
        cmdstring = "rm -rf %s" % (SESSION_DIR+'tmpdir/')
        system(cmdstring)
    else:
        cmdstring = "rsync %s %s" % (srcdir+'*', dicomdir)
        system(cmdstring)
        print(cmdstring)
        system('echo -------copying dicoms to dicom directory from source-------')
        # delete tmpdir
        cmdstring = "rm -rf %s" % (SESSION_DIR+'tmpdir/')
        system(cmdstring)

    # check if scan.info exists (same for all files within session)
    exists = os.path.isfile(SESSION_DIR+'/scan.info') # checks for scan.info file
    if exists==0:
        # unpack these dicoms
        cmdstring = 'unpacksdcmdir -src %s -targ %s -scanonly %s/scan.info' % (srcdir, SESSION_DIR, SESSION_DIR)
        system(cmdstring)
        print(cmdstring)
        system('echo ------unpacking scan info from dicoms-----')
        #make lines low into one function
        
    exists = os.path.isfile(SESSION_DIR+'/scaninfo.csv') # checks for parsed scaninfo.csv file
    if exists==0:
        with open(SESSION_DIR+'/scan.info', 'r') as in_file:
            print("Parsing scan.info to scaninfo.csv ...")
            system('echo -------Parsing scan.info to scaninfo.csv ...-------')
            for line in in_file:
                # make list out of elements in each line
                editline = line.split()
                with open(SESSION_DIR+"/scaninfo.csv", 'a') as result:
                    wr = csv.writer(result, dialect='excel')
                    wr.writerow(editline)
            print("COMPLETED PARSING SCANINFO FILE")
            system('echo ------COMPLETED PARSING SCANINFO FILE------')
            
    # import scan.info from SESSION_DIR as pandas dataframe
    scan_info = pd.read_csv(SESSION_DIR+'/scaninfo.csv', header=None)
    
    # for each of the confirmed runs, make a bold_directory for task
    for element in task:
        # initialize indexing list to later add to cmdstrng
        run_list = []
        scan_name_index = []
        scan_num_index = []
        runsdir = "/bold_"+element
        try:
            os.makedirs(SESSION_DIR+runsdir)
            for row in range(0, len(scan_info.index)):
                # index values of every row with inclusion criteria (obain task name and run #)
                if element in scan_info.iloc[row,1]:
                    # ask for confirmation of task contents (these are all the runs)
                    instring = 'Would you like to include run '+str(scan_info.iloc[row,1])+' for task '+str(element)+' ?'
                    userinput = verifyrun(instring)
                    if userinput == 1:
                        scan_name_index.append(scan_info.iloc[row,1])
                        scan_num_index.append(scan_info.iloc[row,0])

            for name in range(0, len(scan_name_index)):
                print(scan_name_index[name])
                temp = ' -run '+ str(scan_num_index[name]) + ' '+ runsdir+ ' ' + unpackedtype + ' ' + 'f.'+unpackedtype
                run_list.append(temp) 

            #combine list of strings into one long string about run info
            runstring = "".join(run_list)
            cmdstring = 'unpacksdcmdir -src %s -targ %s %s' % (srcdir, SESSION_DIR ,runstring) # directory sturture may be messed up!
            print(cmdstring)
            system('echo ------------unpacking raw functional data--------------')
            system(cmdstring)
        except FileExistsError:
            print("functional directory already exists and is unpacked")
            system('echo -------functional directory already exists and is unpacked------')
        
print("------------------------- UNPACKING COMPLETE --------------------------------")
system('echo ------------------------- UNPACKING COMPLETE --------------------------------')


cp -a /autofs/cluster/animal/users/rezzo/recons/SKRA_M_48_20171018/* /autofs/cluster/animal/users/rezzo/PCA/PCA_SKRA_M_48_20171018/anat/
MR.1.3.12.2.1107.5.2.32.35162.201710181420457300872155
rsync /cluster/archive/319/siemens/TrioTim-35162-20171018-133630-001486/* /autofs/cluster/animal/users/rezzo/PCA/PCA_SKRA_M_48_20171018/dicoms/
unpacksdcmdir -src /cluster/archive/319/siemens/TrioTim-35162-20171018-133630-001486/ -targ /autofs/cluster/animal/users/rezzo/PCA/PCA_SKRA_M_48_20171018/ -scanonly /autofs/cluster/animal/users/rezzo/PCA/PCA_SKRA_M_48_20171018//scan.info
Parsing scan.info to scaninfo.csv ...
COMPLETED PARSING SCANINFO FILE
Would you like to include run fMAP_FPW_3mm_TR2_ISO_TP169 for task FPW ? [y/n]: y
fMAP_FPW_3mm_TR2_ISO_TP169
unpacksdcmdir -src /cluster/archive/319/siemens/TrioTim-35162-20171018-133630-001486/ -targ /autofs/cluster/animal/users/rezzo/PCA/PCA_SKRA_M_48_20171018/  -run 12 /bold_FPW nii f.nii
Would you like to include run fMAP_MT_3mm_TR2_ISO_TP153 for task

0

In [53]:
# # PARADIGM FILE MIGHT ALWAYS BE MANUAL
# # find a way to create paradigm files automatically; for now, manually add

# for elements in range(0, len(session_values["SUBJECT"])): # define range as number of sessions
#     srcdir = session_values["PATH"][elements]+"/"
#     SESSION_DIR = projectdir+"/"+session_values["SUBJECT"][elements]+'/'
#     dicomdir = SESSION_DIR+"dicoms/"
#     taskname = session_values["SUBJECT"][elements]
#     runsdir = SESSION_DIR+"bold_"+projectname+"/"
#     anatdir = SESSION_DIR+"anat/"

#     if 'MT' in taskname:
#         parsrc = '/autofs/cluster/animal/users/rezzo/backup_parfiles/MT_run1_paradigmFile'
#         cmdstring = 'cp %s %s' % (parsrc, runsdir)
#         print(cmdstring)
#         system(cmdstring)
#     else:
#         print("task name needed")


task name needed


In [ ]:
# add name of subject in subname : SKRA
# add symlink "SKRA" to their anat folder connecting to the recon
ln -s link symlinkname # in anat folder

# need way to copy over paradigm files for ech subject




In [83]:
# in terminal::
# FOR MT

# change sessidfile to have the subject of interest

# this one is linked to /space/weck/2/users/MR/ITAY/
setenv SUBJECTS_DIR /autofs/cluster/animal/users/rezzo/recons/ITAY_F_34_20170110/
setenv SUBJECT edit.FS6_004_019

# this one has a recursive symlink
setenv SUBJECTS_DIR /autofs/cluster/animal/users/rezzo/recons/ITAY_F_34_20171220/
setenv SUBJECT edit.FS6_004_006

# WARNING" these create files with specific anatomical calls (make sure to set variable)
mkanalysis-sess -fsd bold -native -analysis firstlevelMT.lh -surface $SUBJECT lh -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 25 -polyfit 2 -nskip 4 -TR 2 -nconditions 2 -force
mkanalysis-sess -fsd bold -native -analysis firstlevelMT.rh -surface $SUBJECT rh -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 25 -polyfit 2 -nskip 4 -TR 2 -nconditions 2 -force
mkanalysis-sess -fsd bold -native -analysis firstlevelMT.whole -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 25 -polyfit 2 -nskip 4 -TR 2 -nconditions 2 -force

mkcontrast-sess -analysis firstlevelMT.rh -contrast moving-vs-still -a 1 -c 2
mkcontrast-sess -analysis firstlevelMT.lh -contrast moving-vs-still -a 1 -c 2
mkcontrast-sess -analysis firstlevelMT.whole -contrast moving-vs-still -a 1 -c 2

# !!!!!!!!!!!!!! REPLACE SUBJECT NAME
preproc-sess -s PCA_ITAY_F_34_20170110 -fsd bold -surface $SUBJECT lhrh -per-run -fwhm 5 -force
preproc-sess -s PCA_ITAY_F_34_20171220 -fsd bold -surface $SUBJECT lhrh -per-run -fwhm 5 -force
preproc-sess -s PCA_SKRA_M_48_20171018 -fsd bold_MT -surface $SUBJECT lhrh -per-run -fwhm 5 -force

# !!!!!!!!!!!!!!!! REPLACE SESSION ID FILE
## needs to run on 'bold'? Also the visualizations do
## make a file with a list of all sessions in project folder
selxavg3-sess -sf sessidfile -analysis firstlevelMT.lh -overwrite
selxavg3-sess -sf sessidfile -analysis firstlevelMT.rh -overwrite
selxavg3-sess -sf sessidfile -analysis firstlevelMT.whole -overwrite

# visualize surface by hemisphere
tksurfer-sess -s PCA_ITAY_F_34_20170110 -analysis firstlevelMT.lh -c moving-vs-still # for surface
tksurfer-sess -s PCA_ITAY_F_34_20170110 -analysis firstlevelMT.rh -c moving-vs-still

# FOR VOLUME JAN
#setenv SUBJECTS_DIR /autofs/cluster/animal/scan_data/dl-xnat/MASTER/ITAY_F_34/PCA_ITAY_F_34_20171220/recons/FS6_004_006/2017-12-21T17-10-11-115/
tkmedit-sess -s PCA_ITAY_F_34_20170110 -analysis firstlevelMT.whole -c moving-vs-still
# FOR VOLUME DEC
#setenv SUBJECTS_DIR /autofs/cluster/animal/scan_data/dl-xnat/MASTER/ITAY_F_34/PCA_ITAY_F_34_20171220/recons/FS6_004_006/2017-12-21T17-10-11-115/
tkmedit-sess -s PCA_ITAY_F_34_20171220 -analysis firstlevelMT.whole -c moving-vs-still

'PCA_ITAY_F_34_20171220'

In [ ]:
# FOR FPW

# change sessidfile to have the subject of interest


# this one has a recursive symlink
setenv SUBJECTS_DIR /autofs/cluster/animal/users/rezzo/recons/ITAY_F_34_20171220/
setenv SUBJECT edit.FS6_004_006

mkanalysis-sess -fsd bold -native -analysis firstlevelFPW.lh -surface $SUBJECT lh -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 8 -polyfit 2 -nskip 4 -TR 2 -nconditions 4 -force
mkanalysis-sess -fsd bold -native -analysis firstlevelFPW.rh -surface $SUBJECT rh -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 8 -polyfit 2 -nskip 4 -TR 2 -nconditions 4 -force
mkanalysis-sess -fsd bold -native -analysis firstlevelFPW.whole -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 8 -polyfit 2 -nskip 4 -TR 2 -nconditions 4 -force

mkcontrast-sess -analysis firstlevelFPW.rh -contrast faces-vs-placeswords -a 1 -c 4 -c 2
mkcontrast-sess -analysis firstlevelFPW.lh -contrast faces-vs-placeswords -a 1 -c 4 -c 2
mkcontrast-sess -analysis firstlevelFPW.whole -contrast faces-vs-placeswords -a 1 -c 4 -c 2

mkcontrast-sess -analysis firstlevelFPW.rh -contrast places-vs-faceswords -a 4 -c 1 -c 2
mkcontrast-sess -analysis firstlevelFPW.lh -contrast places-vs-faceswords -a 4 -c 1 -c 2
mkcontrast-sess -analysis firstlevelFPW.whole -contrast places-vs-faceswords -a 4 -c 1 -c 2

preproc-sess -s PCA_ITAY_F_34_20171220 -fsd bold -surface $SUBJECT lhrh -per-run -fwhm 5 -force

# ADD REGISTATION?


# !!!!!!!!!!!!!!!!1 REPLACE SESSION ID FILE
## make a file with a list of all sessions in project folder
selxavg3-sess -sf sessidfile -analysis firstlevelFPW.lh -overwrite
selxavg3-sess -sf sessidfile -analysis firstlevelFPW.rh -overwrite
selxavg3-sess -sf sessidfile -analysis firstlevelFPW.whole -overwrite

# visualize surface by hemisphere
tksurfer-sess -s PCA_ITAY_F_34_20171220 -analysis firstlevelFPW.lh -c faces-vs-placeswords # for surface
tksurfer-sess -s PCA_ITAY_F_34_20171220 -analysis firstlevelFPW.rh -c faces-vs-placeswords

# FOR VOLUME
#setenv SUBJECTS_DIR /autofs/cluster/animal/scan_data/dl-xnat/MASTER/ITAY_F_34/PCA_ITAY_F_34_20171220/recons/FS6_004_006/2017-12-21T17-10-11-115/
tkmedit-sess -s PCA_ITAY_F_34_20171220 -analysis firstlevelFPW.whole -c faces-vs-placeswords



In [ ]:
# to extract stats from ROI for volume (1 condition)

# this one is linked to /space/weck/2/users/MR/ITAY/
setenv SUBJECTS_DIR /autofs/cluster/animal/users/rezzo/recons/ITAY_F_34_20170110/
setenv SUBJECT edit.FS6_004_019

# # this one has a recursive symlink
# setenv SUBJECTS_DIR /autofs/cluster/animal/users/rezzo/recons/ITAY_F_34_20171220/
# setenv SUBJECT edit.FS6_004_006

setenv CON /autofs/cluster/animal/users/rezzo/PROJECT/PCA_ITAY_F_34_20170110/bold/firstlevel.whole/moving-vs-still/ces.nii.gz
setenv SIG /autofs/cluster/animal/users/rezzo/PROJECT/PCA_ITAY_F_34_20170110/bold/firstlevel.whole/moving-vs-still/sig.nii.gz
setenv REG /autofs/cluster/animal/users/rezzo/PROJECT/PCA_ITAY_F_34_20170110/bold/register.dof6.lta

mri_vol2vol --mov $CON --reg $REG --fstarg --interp nearest --o ces.moving.anat.bb.mgh 
mri_vol2vol --mov $SIG --reg $REG --fstarg --interp nearest --o sig.moving.anat.bb.mgh 

mri_segstats --seg $SUBJECTS_DIR/$SUBJECT/mri/aparc+aseg.mgz --ctab $FREESURFER_HOME/FreeSurferColorLUT.txt --id 1007 --id 1015 --id 2015 --i ces.moving.anat.bb.mgh --sum ces.moving.bb.stats
mri_segstats --seg $SUBJECTS_DIR/$SUBJECT/mri/aparc+aseg.mgz --ctab $FREESURFER_HOME/FreeSurferColorLUT.txt --id 1007 --id 1015 --id 2015 --i ces.moving.anat.bb.mgh --sum ces.moving.abs-masked.bb.stats --mask sig.moving.anat.bb.mgh --maskthresh 2 --masksign abs

